<a href="https://colab.research.google.com/github/GLopezMUZH/Packing_and_Vehcicle_Routing/blob/main/ItemsFromPlants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Knapsack - dynamic programming

In [ ]:
# A Dynamic Programming based Python 
# Program for 0-1 Knapsack problem
# code based on snap: https://codereview.stackexchange.com/questions/220450/python-program-for-0-1-knapsack-problem
def knapsack(capacity, weights, values):
    items = len(weights)

    capacity_items_matrix = [[0] * (capacity + 1)]
    for item_idx in range(items):
        capacity_items_matrix.append(capacity_items_matrix[item_idx].copy())
        for k in range(weights[item_idx], capacity + 1):
            capacity_items_matrix[item_idx + 1][k] = max(capacity_items_matrix[item_idx][k], capacity_items_matrix[item_idx][k -weights[item_idx]] + values[item_idx])

    solution_value = capacity_items_matrix[items][capacity]
    solution_weight = 0
    taken = []
    k = capacity
    for item_idx in range(items, 0, -1):
        if capacity_items_matrix[item_idx][k] != capacity_items_matrix[item_idx - 1][k]:
            taken.append(item_idx - 1)
            k -= weights[item_idx - 1]
            solution_weight += weights[item_idx - 1]

    return solution_value, solution_weight, taken

# Utility functions

In [ ]:
def convertToIntList(someDict):
    w1_arr = []
    for element in someDict:
        w1_arr.append(element[0])
    return w1_arr

In [ ]:
def printSelectedItems(w_arr, p_arr, selectedIndexes):
    plant1_selected_weights = []
    plant1_selected_values = []
    for i in range(len(w_arr)):
        if (i in selectedIndexes):
            plant1_selected_weights.append(w_arr[i])
            plant1_selected_values.append(p_arr[i])

    plant1_selected_weights = np.array(plant1_selected_weights)
    plant1_selected_values = np.array(plant1_selected_values)

    print("Selected weights: ", plant1_selected_weights)
    print("Sum weights: ", np.sum(plant1_selected_weights))

    print("Selected values: ",plant1_selected_values)
    print("Sum values: ", np.sum(plant1_selected_values))

In [ ]:
#remove from w and p arrays
def removeSelected(w_arr, p_arr, selectedIndexes):
    remaining_weights = []
    remaining_values = []
    for i in range(len(w_arr)):
        if (i not in selectedIndexes):
            remaining_weights.append(w_arr[i])
            remaining_values.append(p_arr[i])
    return remaining_weights, remaining_values


# Knapsack example

In [ ]:
values = [60, 100, 120, 200]
weights = [10, 20, 30, 30]
capacity = 50
items = len(values)

print(knapsack(capacity, weights, values))

(300, 50, [3, 1])


# Ex 4 - Production plants items

In [ ]:
# read the input data
file1_dir_path = '/content/gdrive/MyDrive/UZH_varios/ETH_L&GV_Ex4/inputDataAssignment.mat'
file1_dict = loadmat(file1_dir_path)
#print(file1_dict)
# print(file1_dict['p3'])

W_val = file1_dict.get("W")[0][0]
Z_val = file1_dict.get("Z")[0][0]
print("Weight capacity of each truck: ", W_val)
print("Cost of renting a truck: ",Z_val)

w1_arr = convertToIntList(file1_dict['w1'])
w2_arr = convertToIntList(file1_dict['w2'])
w3_arr = convertToIntList(file1_dict['w3'])
p1_arr = convertToIntList(file1_dict['p1'])
p2_arr = convertToIntList(file1_dict['p2'])
p3_arr = convertToIntList(file1_dict['p3'])

Weight capacity of each truck:  100
Cost of renting a truck:  32


In [ ]:
print(w3_arr)
print(p3_arr)


[29, 24, 44, 34, 18, 35, 27, 33, 43, 28, 44, 36, 18, 44, 38, 46, 22, 21, 45, 38, 19, 39, 39, 44, 38, 21, 18, 18, 18, 25, 42, 33, 34, 42, 21, 26, 34, 46, 34, 18, 41, 24, 41, 29, 43, 39, 34, 21, 19, 21, 19, 21, 24, 38, 34, 18, 20, 46, 34, 23]
[10.569829603658471, 18.774047423131897, 13.909062593488523, 21.90127299819715, 17.54130583305118, 30.44315684432855, 9.554506072437102, 18.800240485237875, 29.10938150683943, 23.929745409294256, 12.463998159763552, 5.968431786628731, 3.771338798904101, 24.78930714904499, 25.28444161140257, 29.14209119008142, 9.234677054324406, 20.796798356262215, 28.925254636151728, 17.978559875740675, 11.747813738754033, 30.557836384550544, 28.03507041797307, 16.50799145658907, 7.842615152038662, 9.756002932766796, 12.334677857429936, 5.915662351697045, 14.217159972203852, 5.163900228774942, 15.593249019253912, 27.52336551300084, 10.69025076781417, 29.128753447413864, 12.515365017858647, 24.338256145825653, 12.709276667077505, 9.47907864574533, 26.34340633904255, 

## Plant 1 - 1DKP

In [ ]:
# Calculate for plant 1 - one truck
plant1 = knapsack(W_val, w1_arr, p1_arr)
print(plant1)
print(plant1[2])

(260, 100, [12, 9, 4, 1, 0])
[12, 9, 4, 1, 0]


In [ ]:
printSelectedItems(w1_arr, p1_arr, plant1[2])

Selected weights:  [20 28 13 24 15]
Sum weights:  100
Selected values:  [48 77 33 61 41]
Sum values:  260


## Plant 2- 1DKP in two stages

In [ ]:
# Calculate for plant 2 - two trucks
# solution 2 - calc first truc, reduce arrays, calc 2nd truck
plant2_1 = knapsack(W_val, w2_arr, p2_arr)
print(plant2_1)

(204, 98, [24, 20, 9, 3, 1])


In [ ]:
printSelectedItems(w2_arr, p2_arr, plant2_1[2])

Selected weights:  [29 30 14 13 12]
Sum weights:  98
Selected values:  [61 61 29 28 25]
Sum values:  204


In [ ]:
print(w2_arr)
print(p2_arr)
n_w2_arr, n_p2_arr = removeSelected(w2_arr, p2_arr, plant2_1[2])
print(n_w2_arr)
print(n_p2_arr)

[35, 29, 18, 30, 13, 22, 33, 18, 18, 14, 11, 27, 16, 17, 23, 12, 25, 14, 25, 28, 13, 21, 28, 21, 12, 24, 27]
[46, 61, 26, 61, 8, 14, 61, 20, 13, 29, 11, 47, 27, 34, 35, 21, 26, 12, 50, 33, 28, 33, 43, 13, 25, 29, 39]
[35, 18, 13, 22, 33, 18, 18, 11, 27, 16, 17, 23, 12, 25, 14, 25, 28, 21, 28, 21, 24, 27]
[46, 26, 8, 14, 61, 20, 13, 11, 47, 27, 34, 35, 21, 26, 12, 50, 33, 33, 43, 13, 29, 39]


In [ ]:
plant2_2 = knapsack(W_val, n_w2_arr, n_p2_arr)
print(plant2_2)
printSelectedItems(n_w2_arr, n_p2_arr, plant2_2[2])

(180, 98, [15, 11, 10, 4])
Selected weights:  [33 17 23 25]
Sum weights:  98
Selected values:  [61 34 35 50]
Sum values:  180


In [ ]:
## Plant 3 - 1D Bin-Packing Problem 
# bring all items in as less as possible trucks
print("Min nr of trucks if continuous: ",np.sum(np.array(w3_arr))/W_val)

Min nr of trucks if continuous:  18.65


In [ ]:
# Returns number of bins required using best fit online algorithm
# code based on https://www.geeksforgeeks.org/bin-packing-problem-minimize-number-of-used-bins/
def bestFit(weight, n, c):
     
    # Initialize result (Count of bins)
    res = 0
 
    # Create an array to store
    # remaining space in bins
    # there can be at most n bins
    bin_rem = [0]*n
    trucks_items = [[] for i in range(n)]
    trucks_items_weights = [[] for i in range(n)]
 
    # Place items one by one
    for i in range(n):
        #print("item i= ", i, ", weight = ", weight[i])

        # Find the first bin that can accommodate weight[i]
        j = 0
         
        # Initialize minimum space left and index of best bin
        min = c + 1
        bi = 0
 
        # loop over trucks and update which is the actual minimum rest space, at the end of the comparison one gets the index (bi) of the truck with the minimum rest place
        for j in range(res):
            if (bin_rem[j] >= weight[i] and bin_rem[j] - weight[i] < min):
                bi = j
                min = bin_rem[j] - weight[i]
             
        # If no bin could accommodate weight[i], create a new bin
        if (min == c + 1):
            bin_rem[res] = c - weight[i]
            #print("Create truck. Idx =  ", res, ", item idx = ", i, ", weight = ", weight[i])
            trucks_items[res].append(i)
            trucks_items_weights[res].append(weight[i])
            res += 1
        else: # Assign the item to best bin
            bin_rem[bi] -= weight[i]
            #print("Assign to best truck. Idx =  ", bi, ", item idx = ", i, ", weight = ", weight[i])
            trucks_items[bi].append(i)
            trucks_items_weights[bi].append(weight[i])
    return res, trucks_items, trucks_items_weights
     
# This code is contributed by Rajput-Ji + GLM alterations

In [ ]:
# Driver Code
weight = [2, 5, 4, 7, 1, 3, 8]
c = 10
n = len(weight)
nr_trucks = 0

#nr_trucks, trucks_items = bestFit(weight, n, c)

#print("Number of bins required in Next Fit :", nr_trucks)
#print(trucks_items)


w_sort = np.sort(weight)[::-1]
print(w_sort)
nr_trucks, trucks_items,trucks_items_weights = bestFit(w_sort, n, c)

print("Number of bins required in Next Fit :",
                           nr_trucks)
print(trucks_items)
print(trucks_items_weights)

[8 7 5 4 3 2 1]
Number of bins required in Next Fit : 3
[[0, 5], [1, 4], [2, 3, 6], [], [], [], []]
[[8, 2], [7, 3], [5, 4, 1], [], [], [], []]


In [ ]:
w3_sort = np.sort(w3_arr)[::-1]
print(w3_sort)
n = len(w3_sort)
nr_trucks, trucks_items, trucks_items_weights = bestFit(w3_sort, n, W_val)

print("Number of bins required in Next Fit :", nr_trucks)
print(trucks_items)
print(trucks_items_weights)

[46 46 46 45 44 44 44 44 43 43 42 42 41 41 39 39 39 38 38 38 38 36 35 34
 34 34 34 34 34 34 33 33 29 29 28 27 26 25 24 24 24 23 22 21 21 21 21 21
 21 20 19 19 19 18 18 18 18 18 18 18]
Number of bins required in Next Fit : 20
[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13, 53], [14, 15, 42], [16, 17, 41], [18, 19, 38], [20, 21, 36], [22, 23, 32], [24, 25, 33], [26, 27, 34], [28, 29, 35], [30, 31, 37], [39, 40, 43, 44], [45, 46, 47, 48], [49, 50, 51, 52, 54], [55, 56, 57, 58, 59], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[46, 46], [46, 45], [44, 44], [44, 44], [43, 43], [42, 42], [41, 41, 18], [39, 39, 22], [39, 38, 23], [38, 38, 24], [38, 36, 26], [35, 34, 29], [34, 34, 29], [34, 34, 28], [34, 34, 27], [33, 33, 25], [24, 24, 21, 21], [21, 21, 21, 21], [20, 19, 19, 19, 18], [18, 18, 18, 18, 18], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [ ]:
def getIndexesForWeights(w_arr):
    d = dict(enumerate(np.array(w_arr).flatten(), 0))
    return d


In [ ]:
dict_indexes_weights =  getIndexesForWeights(w3_arr)
trucks_items_indexes = [[] for i in range(nr_trucks)]
val_list = list(dict_indexes_weights.values())

truk_index = 0

for ti in trucks_items_weights:
    if sum(ti) > 0:
        for e in ti:
            position = val_list.index(e)
            dict_indexes_weights[position] = -1
            trucks_items_indexes[truk_index].append(position)
            #print("truck index = ", truk_index, ", weight = ", e, ", pos_in_array= ", position)
            val_list = list(dict_indexes_weights.values())
    truk_index += 1

In [ ]:
for ti in trucks_items_weights:
    if sum(ti) > 0:
        print(ti)


[46, 46]
[46, 45]
[44, 44]
[44, 44]
[43, 43]
[42, 42]
[41, 41, 18]
[39, 39, 22]
[39, 38, 23]
[38, 38, 24]
[38, 36, 26]
[35, 34, 29]
[34, 34, 29]
[34, 34, 28]
[34, 34, 27]
[33, 33, 25]
[24, 24, 21, 21]
[21, 21, 21, 21]
[20, 19, 19, 19, 18]
[18, 18, 18, 18, 18]


In [ ]:
pack3_arr = [[] for i in range(len(trucks_items_indexes))]

i = 0
for ti in trucks_items_indexes:
    #if sum(ti) > 0:
    print(ti)
    pack3_arr[i].append(np.array(ti))
    i += 1

print(pack3_arr)

[15, 37]
[57, 18]
[2, 10]
[13, 23]
[8, 44]
[30, 33]
[40, 42, 4]
[21, 22, 16]
[45, 14, 59]
[19, 24, 1]
[53, 11, 35]
[5, 3, 0]
[32, 36, 43]
[38, 46, 9]
[54, 58, 6]
[7, 31, 29]
[41, 52, 17, 25]
[34, 47, 49, 51]
[56, 20, 48, 50, 12]
[26, 27, 28, 39, 55]
[[array([15, 37])], [array([57, 18])], [array([ 2, 10])], [array([13, 23])], [array([ 8, 44])], [array([30, 33])], [array([40, 42,  4])], [array([21, 22, 16])], [array([45, 14, 59])], [array([19, 24,  1])], [array([53, 11, 35])], [array([5, 3, 0])], [array([32, 36, 43])], [array([38, 46,  9])], [array([54, 58,  6])], [array([ 7, 31, 29])], [array([41, 52, 17, 25])], [array([34, 47, 49, 51])], [array([56, 20, 48, 50, 12])], [array([26, 27, 28, 39, 55])]]


In [ ]:
pack2_arr = [[] for i in range(2)]

pack2_arr[0].append(plant2_1[2])
pack2_arr[1].append(plant2_2[2])

print(pack2_arr)

[[[24, 20, 9, 3, 1]], [[15, 11, 10, 4]]]


In [ ]:
# save to matlab
mdic = {"pack1" : np.array(plant1[2]), "pack2" : pack2_arr,  "pack3" : pack3_arr}
print(mdic)

{'pack1': array([12,  9,  4,  1,  0]), 'pack2': [[[24, 20, 9, 3, 1]], [[15, 11, 10, 4]]], 'pack3': [[array([15, 37])], [array([57, 18])], [array([ 2, 10])], [array([13, 23])], [array([ 8, 44])], [array([30, 33])], [array([40, 42,  4])], [array([21, 22, 16])], [array([45, 14, 59])], [array([19, 24,  1])], [array([53, 11, 35])], [array([5, 3, 0])], [array([32, 36, 43])], [array([38, 46,  9])], [array([54, 58,  6])], [array([ 7, 31, 29])], [array([41, 52, 17, 25])], [array([34, 47, 49, 51])], [array([56, 20, 48, 50, 12])], [array([26, 27, 28, 39, 55])]]}


In [ ]:
target_file_path = '/content/gdrive/MyDrive/UZH_varios/ETH_L&GV_Ex4/solutionGroup1.mat'
savemat(target_file_path, mdic)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
